In [27]:
import sys
#sys.path.insert(1, '/Users/swastik/ophthalmology/Project_Quality_Assurance/From_Bascom/Hold_out/') # noqa
sys.path.insert(1,'/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/QA_Application/training/Non_Distributed') # noqa
from import_packages.dataset_class import Dataset
from import_packages.train_val_to_ids import inference_val_to_ids
from import_packages.checkpoint import load_checkpoint
import os
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from efficientnet_pytorch import EfficientNet

In [28]:
path_dir = '/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Data_models/laced_thunder/df_test.csv' # noqa


In [29]:
df = pd.read_csv(path_dir).iloc[:, 2:]
df

,Excellent,Good,Adequate,Insuff_for_full,Insuff_for_any,labels,Redfree,other
0,0,0,0,1,0,1,NaN,NaN
1,0,0,0,1,0,1,NaN,NaN
2,0,1,0,0,0,0,NaN,NaN
3,0,0,0,1,0,1,NaN,NaN
4,0,0,0,1,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...
759,0,1,0,0,0,0,NaN,NaN
760,0,1,0,0,0,0,NaN,NaN
761,0,0,0,1,0,1,NaN,NaN
762,0,0,1,0,0,0,NaN,NaN


In [30]:
source_dir = '/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Data_Staging/Preprocessed_Combined_Dataset_224_Lanc' # noqa


In [31]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
random.seed(seed)
# CuDA Determinism
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [32]:

# %%
batch_size = 2
partition, labels = inference_val_to_ids(csv_file=path_dir, stratify_columns='labels') # noqa
test_set = Dataset(partition['test_set'],labels,root_dir=source_dir,test_transform=True) # noqa
test_loader = torch.utils.data.DataLoader(test_set,shuffle=True,pin_memory=True, num_workers =4, batch_size=batch_size) # noqa

In [34]:
model_transfer = EfficientNet.from_pretrained('efficientnet-b7')
n_inputs = model_transfer._fc.in_features
model_transfer._fc = nn.Linear(n_inputs, 2)

Loaded pretrained weights for efficientnet-b7


In [7]:
#weights = torch.tensor([0.0008, 0.0001, 0.0003])
optimizer = torch.optim.SGD(model_transfer.parameters(), lr=3e-4)
criterion_transfer = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(
            optimizer,
            patience=4,
            factor=0.1,
            mode='min',
            verbose=True
        )


In [35]:
if torch.cuda.is_available():
    use_cuda = True
else:
    use_cuda = False

In [38]:
model = load_checkpoint(checkpoint_path='/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Data_models/laced_thunder/binary_checkpoint_224.pt',model = model_transfer) # noqa


In [37]:
def accuracy(model, loader,use_cuda=False):
    """Calculate Accuracy."""
    test_accuracy = 0.0
    test_correct = 0.0
    test_total = 0.0
    model.eval()
    for batch_idx, (data, target) in enumerate(loader):
        # print("data shape:", data.shape)
        # print("target shape:", target.shape)
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # update the average validation loss
        output = model(data).squeeze()
        pred = torch.argmax(output, dim=1)
        test_correct += np.sum(np.squeeze(pred.eq(target.view_as(pred)).cpu().numpy())) # noqa
        test_total += data.size(0)
    test_accuracy = 100. * (test_correct/test_total)
    print(test_accuracy)
    return None

In [39]:
accuracy(model, test_loader, use_cuda)

81.67539267015707


In [71]:
for batch_idx, (data, target) in enumerate(test_loader):
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    #print(data.shape)
    output = model(data).squeeze()
    print(output)
    output = torch.nn.functional.softmax(model(data),dim=1).squeeze()
    print(output)
    output = torch.nn.functional.softmax(model(data),dim=0).squeeze()
    print(output)
    pred = torch.argmax(output, dim=1)
    #print(pred.shape)
    print("Next tensor")
    break

tensor([[-0.2003,  0.7608, -0.6045],
        [-1.8646,  0.3946,  1.4379]], grad_fn=<SqueezeBackward0>)
tensor([[0.2497, 0.5569, 0.1935],
        [0.0260, 0.2943, 0.6797]], grad_fn=<SqueezeBackward0>)
tensor([[0.8826, 0.6271, 0.1035],
        [0.1174, 0.3729, 0.8965]], grad_fn=<SqueezeBackward0>)
Next tensor


In [65]:
output = output.detach().numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

0.38250783

In [75]:
a = torch.randn(1, 3)
b = torch.nn.functional.softmax(a,dim=0)
c = torch.nn.functional.softmax(a,dim=1)
print(a,b,c)

tensor([[-0.6921,  0.0951, -0.3143]]) tensor([[1., 1., 1.]]) tensor([[0.2148, 0.4719, 0.3133]])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [43]:
response = open("/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Application/Quality_scanner/gradio_UI/label_names.txt","r") # noqa
labels = [line.rstrip('\n') for line in response]
labels

['Excellent', 'Good_Adequate', 'Insufficient_for_Any_All']